**Task 1 | GPT-2 Model & Checkpoints**

In [1]:
import sys
import os

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.nn import functional as F

#for testing of my implementation
from transformers import GPT2Model, GPT2Config, GPT2Tokenizer

import warnings
from dataclasses import dataclass
from typing import Optional, Tuple, Union


import torch.utils.checkpoint
from torch.cuda.amp import autocast
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

In [2]:
import os
os.environ['CURL_CA_BUNDLE'] = ''

Errors faced in TASK 1:

1.Keys of pre trained model weights were not matching my implementation both in numbers and name so I have to rename and shift them according to huggingface gpt-2 125 parameters.
Facing error in pretraining because of name difference between hugging face gpt2 . I used nano bot as reference but some names are different thats why the issue


In [31]:
"""
I tried to Define tokenized + positonal embedding seperately for our sequence, Here I took default vocab_size  as 50304
because it is used in original gpt2 models at best efficiency & block size as 1024 as maximum sequence length.(num_embeddings). 
We add dropout layer to avoid overfitting, by performing regularization.
( I avoided assertingparameters and inputs as this is a simple model for demonstrating my understanding of transformors architecture)..  



class TP_Embedding(nn.Module):
    
    def __init__(self,config):
        super(TP_Embedding, self).__init__()
        
        self.token_embedding = nn.Embedding(num_embeddings=config.vocab_size, embedding_dim=config.n_embd)
        self.positional_embedding = nn.Embedding(num_embeddings=config.block_size, embedding_dim=config.n_embd) 
        
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        
        positions = torch.arange(0, x.size(1)).unsqueeze(0).repeat(x.size(0), 1).to(x.device)
        
        token_embedded = self.token_embedding(x)
        positional_embedded = self.positional_embedding(positions)
        
        x = self.dropout(positional_embedded)
        
        return x
"""

#had to comment because no. of keys were not matching my implementation for some reason
#so shifted this class methods directly into the GPT 2 model class

NameError: name 'RotaryEmbedding' is not defined

implementation of a multi head self attention layer . I am avoiding pruning for simplicity and time constraint purposes and will try to use a vanilla layer same as nanogpt

In [22]:
class MHSelfAttention(nn.Module):
    def __init__(self,config):
        super(MHSelfAttention, self).__init__()
        
        d_model = config.n_embd
        n_head = config.n_head
        
        bias = config.bias
        dropout= config.dropout
        
        
        assert d_model % n_head == 0
        
        self.n_head = n_head
        self.head_dim = d_model // n_head
        self.b_bias = bias
        self.dropout = config.dropout

        self.c_attn = nn.Linear(3 * d_model,d_model)
        self.c_proj = nn.Linear(d_model, d_model)
        
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)
        
        #adding flash attention referenced from NANOGPT
        
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        batch_size, seq_len, d_model = x.size()
        q = self.query(x).view(batch_size, seq_len, self.n_head, self.head_dim).transpose(1, 2)
        k = self.key(x).view(batch_size, seq_len, self.n_head, self.head_dim).transpose(1, 2)
        v = self.value(x).view(batch_size, seq_len, self.n_head, self.head_dim).transpose(1, 2)
        
        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            out = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            attn_ = q @ k.transpose(-2, -1) / torch.sqrt(torch.tensor(self.head_dim, dtype=torch.float32))
            attn_ = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            attn_weights = F.softmax(attn_, dim=-1)
            attn_weights = self.att_dropout(attn_weights) 
            out = attn_weights @ v
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)
        out = self.projection(out)
        out = self.att_dropout(out)
        return out

PositionwiseFeedForward layer (  as pointwise in assignment)

In [23]:
class PointwiseFeedForward(nn.Module):
    def __init__(self, config):
        super(PointwiseFeedForward, self).__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)  #fc layer
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.dropout)
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)  #projection layer

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        x = self.dropout(x)
        return x


Normalization layer is defined . We can also use Functional's layernorm for this task

In [24]:
class LayerNorm(nn.Module):
    def __init__(self, d_model,bias, eps=1e-5):
        super(LayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(d_model))
        self.bias = nn.Parameter(torch.zeros(d_model)) if bias else none
        self.eps = eps

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)


Defining a data class decorator so that it can handle input values as well as pre trained values properly

In [26]:
from dataclasses import dataclass
@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster


In [27]:
class GPT2Layer(nn.Module):
    def __init__(self, config):
        super(GPT2Layer, self).__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = MHSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = PointwiseFeedForward(config)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.ff(self.norm2(x))
        return x

In [28]:
class GPT2(nn.Module):
    def __init__(self,config):
        super(GPT2, self).__init__()
        
        
        
        
        self.wte = nn.Embedding(config.vocab_size, config.n_embd),
        self.wpe = nn.Embedding(config.block_size, config.n_embd),
        self.drop = nn.Dropout(config.dropout),
        self.h = nn.ModuleList([GPT2Layer(config) for _ in range(config.n_layer)]),
        self.ln_f = LayerNorm(config.n_embd, bias=config.bias),
        
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        
        self.apply(self._init_weights)
        
    
    def forward(self, x):
        
        positions = torch.arange(0, t, dtype=torch.long, device=device) 

        # forward the GPT model itself
        
        tok_emb = self.wte(x) 
        
        pos_emb = self.wpe(pos) 
        
        x = self.drop(tok_emb + pos_emb)
        
        for layer in self.h:
            x = layer(x)
        
        x = self.ln_f(x)
        x = self.lm_head(x)
        return x
    


    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.ones_(module.weight)
            torch.nn.init.zeros_(module.bias)

    
    def load_pretrained_weights(self, model_name='gpt2'):
        # Load pretrained weights from Hugging Face model
        
        config_ = dict(bias= True, n_layer=12, n_head=12, n_embd=768, vocab_size=50257 , block_size= 1024 )
        config = GPTConfig(**config_)
        pretrained_model = GPT2Model.from_pretrained(model_name,resume_download=True)
        
        state_dict = GPT2(config).state_dict()
        sd_keys = state_dict.keys()
        
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')]
        

        state_dict_pretrained = pretrained_model.state_dict()
        sdt_keys = state_dict_pretrained.keys()
        
        
        
        sdt_keys = [k for k in sdt_keys if not k.endswith('.attn.masked_bias')] 
        sdt_keys = [k for k in sdt_keys if not k.endswith('.attn.bias')]
        
        
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        print(sd_keys)
        print(sdt_keys)
        
        for k in sdt_keys:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                # assert state_dict_pretrained[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    state_dict[k].copy_(state_dict_pretrained[k].t())
            else:
                # vanilla copy over the other parameters
               # assert state_dict_pretrained[k].shape == state_dict[k].shape
                with torch.no_grad():
                    state_dict[k].copy_(state_dict_pretrained[k])

        return model
        
        

In [30]:
# Example usage

config= GPTConfig()
my_gpt_model = GPT2(config)

# Load pretrained weights from Hugging Face GPT-2

"""before this model is working but the pretrained weights are having key differences which need to be solved"""
my_gpt_model.load_pretrained_weights()

# Perform a sample prediction
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
input_sequence = tokenizer.encode("Hello, how are you today?", return_tensors="pt")
output_hidden_states = my_gpt_model(input_sequence)

print("Input Sequence:", input_sequence)
print("Output Hidden States Shape:", output_hidden_states.shape)

C:\Users\kesha\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['lm_head.weight']
['wte.weight', 'wpe.weight', 'h.0.ln_1.weight', 'h.0.ln_1.bias', 'h.0.attn.c_attn.weight', 'h.0.attn.c_attn.bias', 'h.0.attn.c_proj.weight', 'h.0.attn.c_proj.bias', 'h.0.ln_2.weight', 'h.0.ln_2.bias', 'h.0.mlp.c_fc.weight', 'h.0.mlp.c_fc.bias', 'h.0.mlp.c_proj.weight', 'h.0.mlp.c_proj.bias', 'h.1.ln_1.weight', 'h.1.ln_1.bias', 'h.1.attn.c_attn.weight', 'h.1.attn.c_attn.bias', 'h.1.attn.c_proj.weight', 'h.1.attn.c_proj.bias', 'h.1.ln_2.weight', 'h.1.ln_2.bias', 'h.1.mlp.c_fc.weight', 'h.1.mlp.c_fc.bias', 'h.1.mlp.c_proj.weight', 'h.1.mlp.c_proj.bias', 'h.2.ln_1.weight', 'h.2.ln_1.bias', 'h.2.attn.c_attn.weight', 'h.2.attn.c_attn.bias', 'h.2.attn.c_proj.weight', 'h.2.attn.c_proj.bias', 'h.2.ln_2.weight', 'h.2.ln_2.bias', 'h.2.mlp.c_fc.weight', 'h.2.mlp.c_fc.bias', 'h.2.mlp.c_proj.weight', 'h.2.mlp.c_proj.bias', 'h.3.ln_1.weight', 'h.3.ln_1.bias', 'h.3.attn.c_attn.weight', 'h.3.attn.c_attn.bias', 'h.3.attn.c_proj.weight', 'h.3.attn.c_proj.bias', 'h.3.ln_2.weight', 'h.3.

KeyError: 'wte.weight'

keys mismatch between gpt2-small and my implementation